In [24]:
import matplotlib as mpl
import json
import numpy as np
import pandas as pd

mpl.use('pgf')

mpl.rcParams.update({
    'pgf.texsystem': 'pdflatex',
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,  
})

from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt

with open('data/processed_things_2024_01_22.json') as f:
    processed_things = json.load(f)
    
green_lengths = []
distances = []
noises = []
green_length_distance_diffs = []

green_length_longer = 0
green_length_shorter = 0
green_length_equal = 0

for thing_name, thing in processed_things.items():
    if thing["TotalCyclesCount"] == 0:
        continue
    if thing["TotalRemovedCycleCount"] / thing["TotalCyclesCount"] > 0.1:
        continue
    for day_idx in range(7):
        for hour_idx in range(24):
            if thing["MedianGreenLengths"][day_idx][hour_idx] == 0:
                continue
            if thing["Metrics"][day_idx][hour_idx] == -1.0 or thing["MedianGreenLengths"][day_idx][hour_idx] ==  -1.0:
                continue
            green_lengths.append(thing["MedianGreenLengths"][day_idx][hour_idx])
            distances.append(thing["Metrics"][day_idx][hour_idx])
            
            diff = thing["MedianGreenLengths"][day_idx][hour_idx] - thing["Metrics"][day_idx][hour_idx]
            if diff > 0:
                green_length_longer += 1
            elif diff < 0:
                green_length_shorter += 1
            else:
                green_length_equal += 1
                
            green_length_distance_diffs.append(diff)
            
total = green_length_longer + green_length_shorter + green_length_equal
longer_ratio = green_length_longer / total
print("Green length longer: ", longer_ratio)
print("Green length longer: ", green_length_longer)
shorter_ratio = green_length_shorter / total
print("Green length shorter: ", shorter_ratio)
print("Green length shorter: ", green_length_shorter)
equal_ratio = green_length_equal / total
print("Green length equal: ", equal_ratio)
print("Green length equal: ", green_length_equal)
                
fig, (ax1, ax2) = plt.subplots(figsize=(5, 5), ncols=2, gridspec_kw={'width_ratios': [1, 0.05]})

H, xedges, yedges = np.histogram2d(distances, green_lengths, bins=[np.linspace(0, 100, 100), np.linspace(0, 100, 100)])
hex_ax = ax1.pcolor(xedges, yedges, H.T, norm=LogNorm(), cmap='jet', rasterized=True, zorder=10)

ax1.set_aspect('equal')
ax1.set_facecolor('black')
ax1.set_yticks(np.arange(0, 101, 5))
ax1.set_xticks(np.arange(0, 101, 5))
labels = [str(i) if i % 20 == 0 else "" for i in range(0, 101, 5)]
ax1.set_yticklabels(labels)
ax1.set_xticklabels(labels)
ax1.set_ylim(0, 100)
ax1.set_xlim(0, 100)
ax1.set_ylabel('Green Length (s)')
ax1.set_xlabel('Cycle Discrepancy (s)')

# ax1.plot([0, 80], [0, 80], color='white')

# ax1.plot([100, 90], [100, 90], color='white')

ax1.axline((0, 0), (100, 100), color='white', linewidth=1, zorder=20, alpha=0.5)

text_background = dict(boxstyle='round', facecolor='black', alpha=1, linewidth=0)
ax1.text(80, 80, f"{int(round(equal_ratio * 100, 0))}%", fontsize=14,
        verticalalignment='center', bbox=text_background, horizontalalignment='center', color='white', zorder=30)
ax1.text(70, 90, f"{int(round(longer_ratio * 100, 0))}%", fontsize=14,
        verticalalignment='center', horizontalalignment='center', color='white', zorder=30)
ax1.text(90, 70, f"{int(round(shorter_ratio * 100, 0))}%", fontsize=14,
        verticalalignment='center', horizontalalignment='center', color='white', zorder=30)


# Add colorbar
cbar = fig.colorbar(hex_ax, cax=ax2)
ax2.set_aspect(4.21)
cbar.set_label('\# Hourly Buckets with Green Length $>$ 0')

plt.subplots_adjust(wspace=0.07, hspace=0.1)

# fig.supxlabel('Cycle-dependent instability (s)', fontsize=10, y=-0.05)

name = "figures/cycle_discrepancy_green_length_heatmap"

plt.savefig(f'{name}.png', bbox_inches='tight', dpi=300)
plt.savefig(f'{name}.pdf', bbox_inches='tight')
plt.savefig(f'{name}.pgf', bbox_inches='tight', dpi=300)


Green length longer:  0.9043904406018063
Green length longer:  2032017
Green length shorter:  0.07049246139905183
Green length shorter:  158385
Green length equal:  0.025117097999141906
Green length equal:  56434
